In [1]:
import pandas as pd
import numpy as np
#import rowgenerators as rg ****commented out for thesis****

from sklearn.feature_selection import VarianceThreshold

scfp = pd.read_stata('p19i6.dta')



# SELECT only rows where there is debt and where white and black
delinquency = 'x3005'
loan_schedule ='x3004'# 1 on time, 5 sometimes late, 0 inapp.
race = 'x6809'
additional_race = 'x6810'
hispanic = 'x7004'

scfp = scfp[(scfp[race] == 1) | (scfp[race] == 2)]
scfp =  scfp[(scfp[loan_schedule] == 1) | (scfp[loan_schedule] == 5)]

# Remove revealing columns
race_revealing = [additional_race, 'x7004']
race_revealing += [col.replace('x', 'j') for col in race_revealing]
sched_revealing = ['x7571', 'x7570', 'x7569', 'x7556', 'x7564', 'x7554','x7553',
                   'x7534', 'x7533', 'x7532', 'x7166', 'x7529', 'x7821', 'x7844',
                   'x7867', 'x7921', 'x7569', 'x7944', 'x7521', 'x7520', 'x7518',
                   'x7517', delinquency, 'j3005', 'j3004']
sched_revealing += [col.replace('x', 'j') for col in sched_revealing]

exclude = sched_revealing#+ race_revealing
X = scfp.copy()
# Eliminate bad columns
#X = scfp.loc[:, [col for col in scfp.columns if col not in exclude]]
X = X.drop(exclude, axis=1)
X = X.fillna(-1)

# Eliminate columns with names starting with 'j'
j_columns = [col for col in X.columns if col[0] == 'j']
X = X.drop(j_columns, axis=1)

# Remove columns with all -1

# instantiate the VarianceThreshold object with a threshold of 0
selector = VarianceThreshold(threshold=0)

# fit the selector on the DataFrame
selector.fit(X)

# get the indices of the non-constant columns
non_constant_cols = selector.get_support(indices=True)

# select the non-constant columns from the original DataFrame
X = X.iloc[:, non_constant_cols]

# print the filtered DataFrame
print(X.shape)



# Change loan schedule to binary
X[loan_schedule] = X[loan_schedule].replace({1: 0, 5: 1})
# # only include where white or black
# X = X[(X[race] == 1) | (X[race] == 2)]
X[race] = X[race].replace({1: 0, 2:1})




(18362, 2494)


In [2]:
print(X[loan_schedule])

5        0
6        0
7        0
8        0
9        0
        ..
28880    0
28881    0
28882    0
28883    0
28884    0
Name: x3004, Length: 18362, dtype: int8


In [58]:
non_cat = 0
for col in X.columns:
    if len(X[col].unique()) > 10:
        non_cat +=1 
print(non_cat)

1017


In [59]:
# Make continuous into range columns
# Define the number of bins
num_bins = 5

# Iterate through columns
for column in X.columns:
    if column == loan_schedule or column == race:
        continue
    if X[column].dtype != 'object' and len(X[column].unique()) > num_bins:
        # Determine bin edges
        bins = pd.qcut(X[column], q=num_bins, labels=False, duplicates='drop') # Potentially would want to do normal distruibution istead of equal bins
        X[column] = bins
    X[column] = X[column].astype('category') # Make every column categorical for now

print(X.shape)
# Get categorical columns
cat_cols = X.select_dtypes(include='category').columns
cat_one_hot = pd.get_dummies(X[cat_cols], drop_first=True)
X = X.drop(cat_cols, axis=1)
X = pd.concat([X, cat_one_hot], axis=1)
print(X.shape)

(18362, 2494)
(18362, 3533)


(18362, 3533)


In [30]:
# # Try normalizatioN????

# # Min-Max normalization
# for col in X.columns:
#     min_val = np.min(X[col])
#     max_val = np.max(X[col])
#     X[col] = (X[col] - min_val) / (max_val - min_val)


# # print("Original data:", data)
# # print("Normalized data:", normalized_data)

In [60]:
# Split X into test and train dfs
from sklearn.model_selection import train_test_split
X_train, X_test = train_test_split(X, test_size=0.2, random_state=42)

In [61]:
# Output test and train to seperate csvs
X_train.to_csv('SCF_train.csv', index=False)
X_test.to_csv('SCF_test.csv', index=False)

In [42]:
# Find number of features
print(X.shape[1])

2494


In [5]:
print(X_train[loan_schedule].value_counts())
print(X_train[race].value_counts())

0    12579
1     2110
Name: x3004, dtype: int64
0    12479
1     2210
Name: x6809, dtype: int64


In [7]:
# figure out how many categorical columns, and how many discrete columns, (categorical if less than 5 discrete values)
non_cat = 0
for col in X_train.columns:
    if len(X_train[col].unique()) > 10:
        non_cat +=1 
print(non_cat)



1296
